## ANOVA

In [ ]:
# 정규성 : 종속 변수는 정규분포를 따른다는 가정
# 등분산성 : 그룹 간의 분산이 동일하다는 가정
# 독립성 : 종속 변수들이 서로 영향을 주고 받으면 안 된다는 가정
# 독립성과 정규성, 등분산성이 충족될 때만 ANOVA 수행
# 따라서 정규성 검정과 등분산성 검정 먼저 진행

#### 독립성 검정

In [ ]:
# 독립성 검정
import matplotlib.pyplot as plt
import seaborn as sns

# 상관 행렬 계산
correlation_matrix = df[df.columns].corr()

# 히트맵 그리기
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

#### 정규성 검정

In [ ]:
# 정규성 검정 (Shapiro-Wilk)
# 귀무가설 : 정규분포를 따른다
from scipy.stats import shapiro

# 클러스터별 feature에 대해 정규성 검정 수행
for cluster_num in range(6):  # 클러스터 번호는 0부터 5까지 총 6개
    cluster_data = df[df['cluster'] == cluster_num]
    
    print(f"\nCluster {cluster_num} - Shapiro-Wilk Test:")
    for column in cluster_data.columns:
        shapiro_test = shapiro(cluster_data[column])
        if shapiro_test.pvalue >= 0.05:
            print(f"{column} - p-value: {shapiro_test.pvalue}")

In [ ]:
# p-value가 유의수준(0.05)보다 작으면 종속 변수가 정규분포를 따르지 않음(정규성 가정 만족x)
# p-value가 작을수록 귀무가설을 기각하고 대립가설을 채택하는 경향
# p-value가 유의수준(0.05)보다 커야 등분산성 가정 만족함

#### 등분산성 검정

In [ ]:
# 각각의 클러스터에 속하는 feature들의 등분산성을 확인하려면
# 각 클러스터에 대해 등분산성 검정을 따로 수행
# -> 해당 클러스터 내에서 feature 분산이 동일한지 여부를 검정할 수 있음

In [ ]:
# 등분산성 검정 - 레빈검정
# 귀무가설 : 등분산성의 특징이 있다
from scipy.stats import bartlett

# 등분산성 검정을 수행할 데이터프레임들
dataframes = [cluster0, cluster1, cluster2, cluster3, cluster4, cluster5]

# 각 데이터프레임에 대해 등분산성 검정 수행
for i, cluster_data in enumerate(dataframes):
    print(f"\nCluster {i} - Bartlett's Test:")
    for column in cluster_data.columns:
        # 등분산성 검정 수행
        bartlett_test = bartlett(*[dataframe[column] for dataframe in dataframes])
        if bartlett_test.pvalue >= 0.05:
            print(f"{column} - p-value: {bartlett_test.pvalue}")


In [ ]:
# # 등분산성 검정 - 레빈검정
# # 귀무가설 : 등분산성의 특징이 있다
# from scipy.stats import levene

# # feature(경제지표)
# dependent_variables = df['feature1']

# # 클러스터(경기국면)
# clusters = df['Cluster'].unique()

# # 각 클러스터에 대해 등분산성 검정 수행
# for cluster in clusters:
#     data_cluster = dependent_variables[df['Cluster'] == cluster]
#     levene_test = levene(data_cluster)
    
#     print(f"Cluster {cluster} - Levene's p-value:", levene_test.pvalue)

In [ ]:
# 등분산성 검정 - 레빈검정
# 귀무가설 : 등분산성의 특징이 있다
from scipy.stats import levene

# features
features = ['feature1', 'feature2', 'feature3']

# 클러스터
clusters = df['Cluster'].unique()

# 각 클러스터에 대해 등분산성 검정 수행
for cluster in clusters:
    print(f"\nCluster {cluster} - levene's Test:")
    for feature in features:
        data_cluster_feature = df[df['Cluster'] == cluster][feature]
        levene_test = levene(data_cluster_feature)
        print(f"{feature} - p-value: {levene_test.pvalue}")

In [ ]:
# p-value가 유의수준(0.05)보다 작으면 그룹 간의 분산이 동일하지 않음(등분산성 가정 만족 x)
# p-value가 작을수록 귀무가설을 기각하고 대립가설을 채택하는 경향
# p-value가 유의수준(0.05)보다 커야 등분산성 가정 만족함 --> ANOVA수행

#### ANOVA 수행

In [ ]:
# 일원분산분석
from scipy.stats import f_oneway

# 각 클러스터에 해당하는 데이터 가져오기
group_A = df[df['Cluster'] == 'A']['경제지표']
group_B = df[df['Cluster'] == 'B']['경제지표']
group_C = df[df['Cluster'] == 'C']['경제지표']

# ANOVA 수행
f_statistic, p_value = f_oneway(group_A, group_B, group_C)

# 결과 출력
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# # 유의수준 설정
# alpha = 0.05

# # P-value와 유의수준 비교
# if p_value < alpha:
#     print("귀무가설을 기각")
# else:
#     print("귀무가설을 기각하지 못했습니다")


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from statsmodels.formula.api import ols

for col in cluster0.columns:
    F, p = stats.f_oneway(cluster0[col],
                    cluster1[col],
                    cluster2[col],
                    cluster3[col],
                    cluster4[col],
                    cluster5[col])
    # print(col)
    # print( 'F-Ratio: {}'.format(F)
    # , 'p-value:{}'.format(p)
    #  , sep = '\n')
    if p <= 0.05:
        print(f"{col} \n F-Ratio: {F} \n p-value: {p}")

In [ ]:
# ANOVA는 그룹 간의 차이가 통계적으로 유의미한 차이인지 판단
# 분산분석 결과를 통해 각 경제 지표들이 클러스터를 설명하는 데 통계적으로 유의미한지를 나타냄
# p-value가 0.05미만이면 통계적으로 유의